In [1]:
%matplotlib inline

import pandas;
import cv2;
import pickle;
import numpy;
import os;
import os.path;

from matplotlib import pyplot;

import tensorflow as tf;

In [2]:
images_dir = "/Users/ravishchawla/workspace/machinelearning/datasets/dogs_vs_cats/";
train_dir = images_dir + "train/";
test_dir = images_dir + "test/";

In [3]:
file_names_p = 'file_names.p';

train_file_names = [];
test_file_names = [];

if os.path.isfile(file_names_p):
    train_file_names, test_file_names = pickle.load(open(file_names_p, 'rb'));
    train_file_names = train_file_names[0];
    test_file_names = test_file_names[0];
    print('dataset loaded');
else:
    for root, folder, file in os.walk(train_dir):
        train_file_names.append(file);

    for root, folder, file in os.walk(test_dir):
        test_file_names.append(file);

    pickle.dump((train_file_names, test_file_names), open(file_names_p, 'wb'));

dataset loaded


In [298]:
def load_images_and_labels(files_dir, image_names, size_sample):
    max_size = [0, 0];
    
    image_set = numpy.random.choice(image_names, size=size_sample);
    
    loaded_images = [];
    labels_list = [];
    
    for fname in image_set:
        img = cv2.imread(files_dir + fname);
        if img.shape[0] > max_size[0]:
            max_size[0] = img.shape[0];
        if img.shape[1] > max_size[1]:
            max_size[1] = img.shape[1];
        
        if 'cat' in fname:
            labels_list.append([0,1]);
        elif 'dog' in fname:
            labels_list.append([1,0]);
        else:
            labels_list.append([0,0]);

        loaded_images.append(img);
    
    print(max_size);
    
    return [loaded_images, labels_list, max_size];

def process_images(images, border_size):
    images_list = [];

    print(len(images))
    for image_pos in range(0, len(images)):
        img = images[image_pos];
        
        width_diff = (border_size[1] - img.shape[1]);
        height_diff = (border_size[0] - img.shape[0]);
    
        width_diffs = [width_diff // 2, width_diff - (width_diff // 2)];
        height_diffs = [height_diff // 2, height_diff - (height_diff // 2)];
        
        bet_img = cv2.copyMakeBorder(img, height_diffs[0], height_diffs[1], width_diffs[0], width_diffs[1], cv2.BORDER_REFLECT);
        
        bw_img = cv2.cvtColor(bet_img, cv2.COLOR_BGR2GRAY);
        
        flat_image = numpy.reshape(bw_img, [bw_img.shape[0] * bw_img.shape[1]]);
        #flat_image = flat_image.astype('float32');
        
        images_list.append(flat_image);
        
    
    images_list = numpy.array(images_list);
    
    return images_list;
#print(images_with_labels);

#Parameters

learning_rate = 0.001;
training_iters = 20000;
batch_size = 128;
display_step = 10;

num_train_images = 250;
num_test_images = 250;

train_and_test_set = numpy.random.choice(train_file_names, num_train_images + num_test_images);

train_images, train_labels, train_size = load_images_and_labels(train_dir, train_and_test_set[:num_train_images], num_train_images);
test_images, test_labels, test_size = load_images_and_labels(train_dir, train_and_test_set[num_train_images:num_train_images+num_test_images], num_test_images);

max_size = [train_size[0] if train_size[0] > test_size[0] else test_size[0], \
            train_size[1] if train_size[1] > test_size[1] else test_size[1]];


train_images = process_images(train_images, max_size);
test_images = process_images(test_images, max_size);

n_input = max_size[0] * max_size[1]; #data input
n_classes = 2; #total classes = 10
dropout = 0.75; #probability to keep units

x = tf.placeholder(tf.float32, [None, n_input]);
y = tf.placeholder(tf.float32, [None, n_classes]);
keep_prob = tf.placeholder(tf.float32) #dropout

print(max_size);

#train_images, train_labels = process_images_and_labels(train_dir, train_file_names, 5);
#test_images, test_labels = process_images_and_labels(test_dir, test_file_names, 250);

[500, 500]
[500, 500]
250
250
[500, 500]


In [299]:
weights = {
    #5x5 convolution. 1 input. 32 outputs.
    'wc1' : tf.Variable(tf.random_normal([5,5,1,32])),
    
    #max pooling layer
    'mp1' : tf.constant(2),
    
    #5x5 convolution. 32 inputs. 64 outputs.
    'wc2' : tf.Variable(tf.random_normal([5, 5, 32, 64])),
    
    #max pooling layer
    #'mp2' : tf.constant(2),
    
    #reshape from input to output
    'wd1' : tf.Variable(tf.random_normal([max_size[0] * max_size[1] *4 , n_classes])),
    
    #32 inputs. 2 outputs (class prediction).
    'out' : tf.Variable(tf.random_normal([max_size[0] * max_size[1] *4,n_classes]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [300]:
def get_size(ten):
    tens = ten.get_shape();
    val = tens[1];
    for i in range(2, len(tens)):
        val = val * tens[i];
    return val;

def conv2d(x, weights, biases, strides=1):
    x = tf.nn.conv2d(x, weights, strides=[1, strides, strides, 1], padding='SAME');
    x = tf.nn.bias_add(x, biases);
    return tf.nn.relu(x);

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME');

def conv_net(x, weights, biases, dropout):
    #reshape input picture

    x = tf.reshape(x, shape=[-1, max_size[0], max_size[1], 1]);
    
    #convolution layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1']);
    print('conv1  ', conv1.get_shape(), get_size(conv1));
    
    conv1 = maxpool2d(conv1, k=2);
    print('pool1 ', conv1.get_shape(), get_size(conv1));
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2']);
    print('conv2 ', conv2.get_shape(), get_size(conv2));
    
    conv2 = maxpool2d(conv2, k=2);
    print('pool2 ', conv2.get_shape());
        
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]]);
    print('fc1 ', fc1.get_shape(), get_size(fc1));

    #output, class prediction
    matrx = tf.matmul(fc1, weights['out']);
    out = tf.add(matrx, biases['out'])
    
    return out;

In [301]:
pred = conv_net(x, weights, biases, keep_prob);

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y));
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost);

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1));
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32));

init = tf.initialize_all_variables();

conv1   (?, 500, 500, 32) 8000000
pool1  (?, 250, 250, 32) 2000000
conv2  (?, 250, 250, 64) 4000000
pool2  (?, 125, 125, 64)
fc1  (?, 1000000) 1000000


96 x (500 * 500) x 1

(5x5 convolution into:)

96 x 500 x 500 x 32

(pool to keep only half:)

96 x 250 x 250 x 32

(5x5 convolution into:)

96 x 250 x 250 x 64

(pool to keep only half:)

96 x 125 x 125 x 64

(flatten to output:)

96 x 125 x 125 x 64


In [ ]:
with tf.Session() as session:
    session.run(init);
    batch_length = 5;
    
    for batch_pos in range(0, len(train_images), batch_length):
        batch_x = train_images[batch_pos : batch_pos + batch_length];
        batch_y = train_labels[batch_pos : batch_pos + batch_length];
        #print(batch_x.shape, batch_y.shape);
        session.run(optimizer, feed_dict={x: batch_x, y:batch_y});
        
        if batch_pos % 10 == 0:
            print('completed on batch ', batch_pos);
    
    print('optimization finished');
    
    print('Testing accuracy: ', \
         session.run(accuracy, feed_dict={x: test_images, y:test_labels, keep_prob: 1}));


    session.close();

completed on batch  0
completed on batch  10
completed on batch  20
completed on batch  30
completed on batch  40
completed on batch  50
completed on batch  60
completed on batch  70
completed on batch  80
completed on batch  90
completed on batch  100
completed on batch  110
completed on batch  120
completed on batch  130
completed on batch  140
completed on batch  150


In [247]:
with tf.Session() as sess:
    writer = tf.train.SummaryWriter("classes", sess.graph);
    writer.close();

sess.close()

In [181]:
batch_length = 5;
for batch_pos in range(0, len(train_images), batch_length):
    batch_x = train_images[batch_pos : batch_pos + batch_length];
    batch_y = train_labels[batch_pos : batch_pos + batch_length];
    print(batch_x.shape);

train_images.shape

(5, 250000)


(5, 250000)

In [177]:
max_size
480*500

240000

In [146]:
t1 = 5;
t2 = 5;

dat = numpy.array([1,2,3,4,5,6,7,8,9,10,11]);

print(dat[:t1]);
print(dat[t1:t1+t2]);

[1 2 3 4 5]
[ 6  7  8  9 10]
